In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import openpyxl
from sklearn.metrics import precision_score, recall_score, f1_score

## Creating the two dictionaries for updated classifier
Note that this is not the final version of the categorisation, this is in the file labelled 2.3.


In [3]:
# read in this file df.to_csv('/Users/emmastoklundlee/Documents/Social Data Science - Masters/Thesis/Idea creation/Football/data/guardian_data_cleaned.csv', index=False)
guardian = pd.read_csv('/Users/emmastoklundlee/Documents/Social Data Science - Masters/Thesis/Idea creation/Football/data/guardian_data_cleaned.csv')

In [4]:
guardian.head()

,type,sectionName,webTitle,webUrl,headline,article_text,word_count,author,char_count,date,article_text_split,womens_football,year,month
0,article,Football,Jürgen Klopp says Mohamed Salah is even better...,https://www.theguardian.com/football/2017/dec/...,Jürgen Klopp says Mohamed Salah is even better...,Jürgen Klopp has said Mohamed Salah exceeds al...,663,Andy Hunter,3875,2017-12-31,"['Jürgen', 'Klopp', 'has', 'said', 'Mohamed', ...",False,2017,12
1,article,Football,‘Unlucky’ Manchester United to regroup after s...,https://www.theguardian.com/football/2017/dec/...,‘Unlucky’ Manchester United to regroup after s...,José Mourinho is becoming ever more desperate ...,775,Jamie Jackson,4385,2017-12-31,"['José', 'Mourinho', 'is', 'becoming', 'ever',...",False,2017,12
2,article,Football,Arsène Wenger attacks refereeing standards aft...,https://www.theguardian.com/football/2017/dec/...,Arsène Wenger attacks refereeing standards aft...,Arsène Wenger delivered a withering attack on ...,415,Stuart James,2322,2017-12-31,"['Arsène', 'Wenger', 'delivered', 'a', 'wither...",False,2017,12
3,article,Football,Manchester United’s Ashley Young given three-m...,https://www.theguardian.com/football/2017/dec/...,Manchester United’s Ashley Young given three-m...,Ashley Young will miss Manchester United’s nex...,276,Guardian sport,1656,2017-12-31,"['Ashley', 'Young', 'will', 'miss', 'Mancheste...",False,2017,12
4,liveblog,Football,West Brom 1-1 Arsenal: Premier League – as it ...,https://www.theguardian.com/football/live/2017...,West Brom 1-1 Arsenal: Premier League – as it ...,Hector Bellerin and Petr Cech engage in a long...,2310,Jacob Steinberg,13126,2017-12-31,"['Hector', 'Bellerin', 'and', 'Petr', 'Cech', ...",False,2017,12


In [5]:
len(guardian)

29865

In [6]:
# Drop rows with NaN values in 'article_text' or 'webTitle'
guardian = guardian.dropna(subset=['article_text', 'webTitle'])


In [20]:
women_dictionary = ["Women’s", "NWSL", "England Women", "SheBelieves Cup", "Continental Cup", "USWNT", "Mathildas", "Lionesses",
"Women’s Super League", "WSL", "Women’s Champions League", "Women’s FA Cup", "Continental Cup", "World Cup 2019", "World Cup 2023", "Euro 2017", "Euro 2022", "Euro 2025", "W-league", 
"Women’s World Cup", "Women’s Euros", "A-League Women", "Weigman", "Hayes", "Eidevall", "Gary Neville", "Gustavsson", "Joe Montemurro", "Eni Aluko", "Karen Carney", "Lucy Bronze", "Rapinoe", 
"Sam Kerr", "Miedema", "Ellen White", "Alex Morgan", "Casey Stoney", "Alexia Putellas", "Beth Mead", "Mary Earps", "Keira Walsh", "Kim Little", "Jenni Hermoso"]

In [21]:
men_dictionary = ["Premier League", "Carabao Cup", "World Cup 2018", "World Cup 2022", "Euro 2020", "Euro 2021", "Euro 2024", "Qatar 2022", "Serie A", "La Liga", 
                  "Ligue 1", "Socceroos", "2018 World Cup", "2022 World Cup", "Europa League", "League One", "League Two", "MLS", "Borussia Dortmund", "Euro 2028", 
                  "Africa Cup of Nations", "Conference League", "A-League Men", "USMNT", "World Cup 2026", "Afcon", "Asian Cup",
                  "Klopp", "Wenger", "Arteta", "Mourinho", "Pochettino", "Ten Hag", "Solsjær", "Southgate", "Lampard", "Bielsa", "Guardiola", "Conte", "Moyes", 
                  "Postecoglou", "Steve Cooper", "Steve Bruce", "Michael O’Neill", "Tuchel", "Xavi", "Maresca", "Dyche", "Ancelotti", "Lee Carsley", "Sven-Göran Eriksson", 
                  "Eddie Howe", "Hjulmand", "Steve Clarke", "Arne Slot", "Ranieri",
                  "Salah", "Saka", "Haaland", "Messi", "Maradona", "Ronaldo", "Lewandowski", "Rashford", "Sterling", "Pogba", "De Bruyne", "Ibrahimovic", "Cahill", "Rooney", 
                  "Mbappé", "Aubameyang", "Pulisic", "Havertz", "Bobby Charlton", "Griezmann", "Bellingham", "Calvert-Lewin", "Beckham", "Harry Kane", "Alan Shearer", "Gary Lineker", 
                  "Rodri", "Lamine Yamal", "Toni Kroos", "Vinícius Júnior", "Beckenbauer"]

In [22]:
# Create the new column 'womens_dictionary'
guardian['womens_dictionary'] = guardian.apply(
    lambda row: any(word.lower() in row['article_text'].lower() for word in women_dictionary) or 
                 any(word.lower() in row['webTitle'].lower() for word in women_dictionary) 
                 if isinstance(row['article_text'], str) or isinstance(row['webTitle'], str) 
                 else False,
    axis=1
)

In [23]:
guardian['womens_dictionary'].value_counts()

womens_dictionary
False    23400
True      6332
Name: count, dtype: int64

In [24]:
guardian['mens_dictionary'] = guardian.apply(
    lambda row: any(word.lower() in row['article_text'].lower() for word in men_dictionary) or 
                 any(word.lower() in row['webTitle'].lower() for word in men_dictionary) 
                 if isinstance(row['article_text'], str) or isinstance(row['webTitle'], str) 
                 else False,
    axis=1
)

In [25]:
guardian['mens_dictionary'].value_counts()

mens_dictionary
True     26090
False     3642
Name: count, dtype: int64

In [26]:
# create new column where guardian['mens_dictionary']= True and guardian['womens_dictionary']=True
guardian['men_and_women'] = guardian['mens_dictionary'] & guardian['womens_dictionary'] 

In [27]:
guardian['men_and_women'].value_counts()

men_and_women
False    25356
True      4376
Name: count, dtype: int64

In [28]:
# Create the new column 'both_false' which will be True where both men_dictionary and women_dictionary are False
guardian['other'] = ~guardian['mens_dictionary'] & ~guardian['womens_dictionary']


In [29]:
guardian['other'].value_counts()

other
False    28046
True      1686
Name: count, dtype: int64

In [30]:
# Create the new column 'both_false' which will be True where both men_dictionary and women_dictionary are False
for title in guardian[guardian['other']]['webTitle'].head(1000):
    print(title)


Nottingham Forest sack Mark Warburton as owners lose confidence
The complete statistics behind our list of 2017's best footballers
Golden Goals: when George Camsell scored five on Christmas Day 1926 | Simon Burnton
Rafael Benítez backs Newcastle players’ decision to cancel Christmas party
Marco Polo del Nero, Brazil football president, handed 90-day Fifa ban
Jamaal Lascelles: Newcastle given home truths in dressing room after latest loss
Crystal Palace fans get police apology over claim they took weapons to Brighton
Manchester United’s 12-year live FA Cup run comes to an end at 58 games
Revealed: why Match of the Day is biased against [insert your club’s name here] | Marina Hyde
Léo Bonatini downs Birmingham to give Wolves four-point lead at the top
Liverpool face Everton in FA Cup third round draw – as it happened
Rainbow flags allowed at World Cup, but LGBT fans cannot ‘propagate’ to minors
Fifa’s new broom Gianni Infantino exhibiting signs of limited shelf life | Marina Hyde
Herbert

In [31]:
# Create the new column 'both_false' which will be True where both men_dictionary and women_dictionary are False
for title in guardian[guardian['men_and_women']]['webTitle'].head(1000):
    print(title)

West Brom 1-1 Arsenal: Premier League – as it happened
That Rhian Brewster issued a wake-up call is both worrying and inspiring | Daniel Taylor
‘I used to say to clubs Virgil van Dijk had it all, and then they’d buy someone else. I’d despair’
Sam Kerr: 'Everyone wants to jump on board and be a Matildas fan'
Newcastle United 0-1 Manchester City: Premier League – as it happened
No more freezing against Iceland: how England are hot on fortifying the mind | Paul MacInnes
World Cup qualification to FFA crisis: Australian football's 'weird' year
Liverpool’s title race is run but we are improving, insists Jürgen Klopp
Watford’s Sebastian Prödl: ‘I’m interested in furniture, interiors and especially kitchens’
Did Lapland really win the World Cup? | The Knowledge Christmas special
The Fiver | 'Capable of doing the job at any club in the world'
How the Guardian ranked the world’s top 100 footballers for 2017
Newcastle Jets take win over Adelaide after controversial handball call - as it happened

In [32]:
# save the dataframe to a new csv file
guardian.to_csv('/Users/emmastoklundlee/Documents/Social Data Science - Masters/Thesis/Idea creation/Football/data/guardian_data_classified.csv', index=False)

In [33]:
# create subset of the dataframe with 500 rows
guardian_subset = guardian.sample(n=500, random_state=1)

In [34]:
guardian_subset.head()

,type,sectionName,webTitle,webUrl,headline,article_text,word_count,author,char_count,date,article_text_split,womens_football,year,month,womens_dictionary,mens_dictionary,men_and_women,other
27676,article,Football,‘Perfect computer’ Rodri is linchpin for Spain...,https://www.theguardian.com/football/article/2...,‘Perfect computer’ Rodri is linchpin for Spain...,"Rodri Hernández stopped, which was when Spain ...",890,Sid Lowe in Cologne,4767,2024-07-01,"['Rodri', 'Hernández', 'stopped,', 'which', 'w...",False,2024,7,False,True,False,False
27636,article,Football,From Greece to Argentina: how tournaments have...,https://www.theguardian.com/football/article/2...,From Greece to Argentina: how tournaments have...,Euro 2004: Greece Otto Rehhagel’s Greece clinc...,1456,Louise Taylor,9320,2024-07-04,"['Euro', '2004:', 'Greece', 'Otto', 'Rehhagel’...",False,2024,7,False,True,False,False
5813,article,Football,'We're over it': Carter-Vickers eager to take ...,https://www.theguardian.com/football/2018/jul/...,'We're over it': Carter-Vickers eager to take ...,"In Earnie Stewart’s full-to-bursting inbox, re...",804,Chris Young,4838,2018-07-26,"['In', 'Earnie', 'Stewart’s', 'full-to-burstin...",False,2018,7,False,True,False,False
8646,article,Football,Concern over gambling branding on children's p...,https://www.theguardian.com/football/2019/dec/...,Concern over gambling branding on children's p...,Football clubs’ matchday programmes are satura...,803,Rob Davies,5053,2019-12-13,"['Football', 'clubs’', 'matchday', 'programmes...",False,2019,12,False,True,False,False
13055,article,Football,Premier League aims for EFL bailout but some c...,https://www.theguardian.com/football/2020/oct/...,Premier League aims for EFL bailout but some c...,The Premier League will attempt to finalise a ...,809,David Hytner and Paul MacInnes,4784,2020-10-13,"['The', 'Premier', 'League', 'will', 'attempt'...",False,2020,10,False,True,False,False


In [35]:
# save the dataframe to a new csv file
guardian_subset.to_csv('/Users/emmastoklundlee/Documents/Social Data Science - Masters/Thesis/Idea creation/Football/data/guardian_data_classified_subset.csv', index=False)

## Checking accuracy of updated dictionaries

In [7]:
# read in the csv file
guardian_annotated = pd.read_csv('/Users/emmastoklundlee/Documents/Social Data Science - Masters/Thesis/Idea creation/Football/data/guardian_data_classified_subset_annotated.csv')

/var/folders/3m/k0nn68mj0ylfjxzs4mcl9_f00000gn/T/ipykernel_64475/3629311433.py:2: DtypeWarning: Columns (1418,1419,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463,1464,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474,1475,1476,1477,1478,1479,1480,1481,1482,1483,1484,1485,1486,1487,1488,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499,1500,1501,1502,1503,1504,1505,1506,1507,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,

In [8]:
# drop columns with unnamed: 
guardian_annotated = guardian_annotated.loc[:, ~guardian_annotated.columns.str.contains('^Unnamed')]

In [9]:
guardian_annotated.head()

,type,sectionName,webTitle,webUrl,headline,article_text,word_count,author,char_count,date,article_text_split,womens_football,year,month,womens_dictionary,mens_dictionary,men_and_women,other,Annotation
0,article,Football,‘Perfect computer’ Rodri is linchpin for Spain...,https://www.theguardian.com/football/article/2...,‘Perfect computer’ Rodri is linchpin for Spain...,"Rodri Hernández stopped, which was when Spain ...",890,Sid Lowe in Cologne,4767,01/07/2024,"['Rodri', 'Hernández', 'stopped,', 'which', 'w...",FALSE,2024,7,FALSE,TRUE,FALSE,FALSE,2
1,article,Football,From Greece to Argentina: how tournaments have...,https://www.theguardian.com/football/article/2...,From Greece to Argentina: how tournaments have...,Euro 2004: Greece Otto Rehhagel’s Greece clinc...,1456,Louise Taylor,9320,04/07/2024,"['Euro', '2004:', 'Greece', 'Otto', 'Rehhagel’...",FALSE,2024,7,FALSE,TRUE,FALSE,FALSE,2
2,article,Football,'We're over it': Carter-Vickers eager to take ...,https://www.theguardian.com/football/2018/jul/...,'We're over it': Carter-Vickers eager to take ...,"In Earnie Stewart’s full-to-bursting inbox, re...",804,Chris Young,4838,26/07/2018,"['In', 'Earnie', 'Stewart’s', 'full-to-burstin...",FALSE,2018,7,FALSE,TRUE,FALSE,FALSE,2
3,article,Football,Concern over gambling branding on children's p...,https://www.theguardian.com/football/2019/dec/...,Concern over gambling branding on children's p...,Football clubs’ matchday programmes are satura...,803,Rob Davies,5053,13/12/2019,"['Football', 'clubs’', 'matchday', 'programmes...",FALSE,2019,12,FALSE,TRUE,FALSE,FALSE,3
4,article,Football,Premier League aims for EFL bailout but some c...,https://www.theguardian.com/football/2020/oct/...,Premier League aims for EFL bailout but some c...,The Premier League will attempt to finalise a ...,809,David Hytner and Paul MacInnes,4784,13/10/2020,"['The', 'Premier', 'League', 'will', 'attempt'...",FALSE,2020,10,FALSE,TRUE,FALSE,FALSE,2


In [10]:
# count values of Annotation column
guardian_annotated['Annotation'].value_counts()

Annotation
2                                                                                                                                                                                                                                                                                               412
1                                                                                                                                                                                                                                                                                                59
3                                                                                                                                                                                                                                                                                                21
0                                                                                                                

In [11]:
# drop rows where 'Annotation' isn't 0, 1, 2 or 3
guardian_annotated = guardian_annotated[guardian_annotated['Annotation'].isin(['0', '1', '2', '3'])]

In [12]:
# check number of true mens_dictionary are 2 in Annotation column
guardian_annotated[guardian_annotated['Annotation'] == '2']['mens_dictionary'].value_counts()

mens_dictionary
TRUE     359
FALSE     29
Name: count, dtype: int64

In [13]:
# create new column for men_annotation where 'Annotation' is 2 display True
guardian_annotated['men_annotation'] = guardian_annotated['Annotation'] == '2'


In [14]:
# create new column for men_annotation where 'Annotation' is 2 display True
guardian_annotated['women_annotation'] = guardian_annotated['Annotation'] == '1'

In [15]:
guardian_annotated.head()

,type,sectionName,webTitle,webUrl,headline,article_text,word_count,author,char_count,date,...,womens_football,year,month,womens_dictionary,mens_dictionary,men_and_women,other,Annotation,men_annotation,women_annotation
0,article,Football,‘Perfect computer’ Rodri is linchpin for Spain...,https://www.theguardian.com/football/article/2...,‘Perfect computer’ Rodri is linchpin for Spain...,"Rodri Hernández stopped, which was when Spain ...",890,Sid Lowe in Cologne,4767,01/07/2024,...,FALSE,2024,7,FALSE,TRUE,FALSE,FALSE,2,True,False
1,article,Football,From Greece to Argentina: how tournaments have...,https://www.theguardian.com/football/article/2...,From Greece to Argentina: how tournaments have...,Euro 2004: Greece Otto Rehhagel’s Greece clinc...,1456,Louise Taylor,9320,04/07/2024,...,FALSE,2024,7,FALSE,TRUE,FALSE,FALSE,2,True,False
2,article,Football,'We're over it': Carter-Vickers eager to take ...,https://www.theguardian.com/football/2018/jul/...,'We're over it': Carter-Vickers eager to take ...,"In Earnie Stewart’s full-to-bursting inbox, re...",804,Chris Young,4838,26/07/2018,...,FALSE,2018,7,FALSE,TRUE,FALSE,FALSE,2,True,False
3,article,Football,Concern over gambling branding on children's p...,https://www.theguardian.com/football/2019/dec/...,Concern over gambling branding on children's p...,Football clubs’ matchday programmes are satura...,803,Rob Davies,5053,13/12/2019,...,FALSE,2019,12,FALSE,TRUE,FALSE,FALSE,3,False,False
4,article,Football,Premier League aims for EFL bailout but some c...,https://www.theguardian.com/football/2020/oct/...,Premier League aims for EFL bailout but some c...,The Premier League will attempt to finalise a ...,809,David Hytner and Paul MacInnes,4784,13/10/2020,...,FALSE,2020,10,FALSE,TRUE,FALSE,FALSE,2,True,False


In [16]:
# value count of men_annotation
guardian_annotated['mens_dictionary'].value_counts()

mens_dictionary
TRUE     404
FALSE     69
Name: count, dtype: int64

In [17]:
guardian_annotated['mens_dictionary'] = guardian_annotated['mens_dictionary'].map({'TRUE': True, 'FALSE': False}).astype(bool)


In [18]:
# Convert boolean columns to binary (0 and 1)
guardian_annotated['mens_dictionary'] = guardian_annotated['mens_dictionary'].astype(int)
guardian_annotated['men_annotation'] = guardian_annotated['men_annotation'].astype(int)

In [19]:


# Calculate precision, recall, and F1 score
precision = precision_score(guardian_annotated['men_annotation'], guardian_annotated['mens_dictionary'], zero_division=0)
recall = recall_score(guardian_annotated['men_annotation'], guardian_annotated['mens_dictionary'], zero_division=0)
f1 = f1_score(guardian_annotated['men_annotation'], guardian_annotated['mens_dictionary'], zero_division=0)

# Output the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.8906976744186047
Recall: 0.9296116504854369
F1 Score: 0.9097387173396676


/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_s

In [20]:
# check number of true womens_dictionary are 1 in Annotation column
guardian_annotated[guardian_annotated['Annotation'] == '1']['womens_dictionary'].value_counts()

womens_dictionary
TRUE     54
FALSE     4
Name: count, dtype: int64

In [21]:
guardian_annotated['womens_dictionary'] = guardian_annotated['womens_dictionary'].map({'TRUE': True, 'FALSE': False}).astype(bool)

In [22]:
# Convert boolean columns to binary (0 and 1)
guardian_annotated['womens_dictionary'] = guardian_annotated['womens_dictionary'].astype(int)
guardian_annotated['women_annotation'] = guardian_annotated['women_annotation'].astype(int)

In [23]:
# Calculate precision, recall, and F1 score
precision = precision_score(guardian_annotated['women_annotation'], guardian_annotated['womens_dictionary'], zero_division=0)
recall = recall_score(guardian_annotated['women_annotation'], guardian_annotated['womens_dictionary'], zero_division=0)
f1 = f1_score(guardian_annotated['women_annotation'], guardian_annotated['womens_dictionary'], zero_division=0)

# Output the results
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# capturing a lot of the postive womens dictionary which is good 

Precision: 0.4166666666666667
Recall: 0.9322033898305084
F1 Score: 0.5759162303664921


/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_s

In [24]:
# check number of men_and_women are 3 in Annotation column
guardian_annotated[guardian_annotated['Annotation'] == '3']['men_and_women'].value_counts()

men_and_women
FALSE    11
TRUE      9
Name: count, dtype: int64

In [25]:
# check number of other are 0 in Annotation column
guardian_annotated[guardian_annotated['Annotation'] == '0']['other'].value_counts()

other
FALSE    7
Name: count, dtype: int64

## the following is based on annotation of the original dictionary that i created

In [4]:
# read in the annotated data
df = pd.read_excel('guardian_subset_annotated.xlsx')

In [15]:
# create subset with first 500 rows
df_subset = df.head(500)

In [16]:
df_subset.head()

,type,sectionName,webTitle,webUrl,headline,article_text,womens_football,word_count,author,char_count,date,article_text_split,womens_football.1,year,month,before_july_2022,month_year,live_blog
0,article,Football,UK government's mixed messages casts shadow ov...,https://www.theguardian.com/football/2020/jun/...,UK government's mixed messages fail to diminis...,The last time English professional football go...,0.0,805,David Conn,4973,2020-06-17,"['The', 'last', 'time', 'English', 'profession...",False,2020,6,1,2020-06-01,0
1,liveblog,Football,"Antony to be available for Manchester United, ...",https://www.theguardian.com/football/live/2023...,"Antony to be available for Manchester United, ...","Here’s Football Daily, to which you can sign u...",0.0,4625,John Brewin (later) & Yara El-Shaboury (earlier),25827,2023-09-29,"['Here’s', 'Football', 'Daily,', 'to', 'which'...",True,2023,9,0,2023-09-01,1
2,article,Football,Tall tales: how my local and I became part of ...,https://www.theguardian.com/football/2023/jul/...,Tall tales: how my local and I became part of ...,"From time to time in this job, an offer of ext...",0.0,1000,Barry Glendenning,5892,2023-07-01,"['From', 'time', 'to', 'time', 'in', 'this', '...",False,2023,7,0,2023-07-01,0
3,article,Football,"Football League: Gayle double sinks Stoke, Joz...",https://www.theguardian.com/football/2018/sep/...,"Football League: Gayle double sinks Stoke, Joz...",Championship Stoke’s troubled start to life in...,0.0,691,John Brewin,4182,2018-09-01,"['Championship', 'Stoke’s', 'troubled', 'start...",False,2018,9,1,2018-09-01,0
4,article,Football,'Something needs to change': Willian the lates...,https://www.theguardian.com/football/2021/feb/...,'Something needs to change': Willian the lates...,Willian has revealed that he is the latest foo...,0.0,426,Nick Ames,2655,2021-02-19,"['Willian', 'has', 'revealed', 'that', 'he', '...",False,2021,2,1,2021-02-01,0


In [17]:
# ensure that womens_football is an integer
df_subset['womens_football'].value_counts()

womens_football
0.0    435
1.0     64
Name: count, dtype: int64

In [11]:
# Drop rows with NaN values in 'womens_football' and convert to int
df_subset = df_subset.dropna(subset=['womens_football'])
df_subset['womens_football'] = df_subset['womens_football'].astype(int)


In [12]:
df_subset['womens_football'].value_counts()

womens_football
0    435
1     64
Name: count, dtype: int64

In [13]:
# compare women_football to womens_football.1 where 0 is False and 1 is True
df_subset['womens_football.1'].value_counts()

womens_football.1
False    440
True      59
Name: count, dtype: int64

In [19]:
# convert womens_football.1 to int
df_subset['womens_football.1'] = df_subset['womens_football.1'].astype(int)

In [20]:
df_subset['womens_football.1'].value_counts()

womens_football.1
0    440
1     59
Name: count, dtype: int64

In [21]:
df_subset['manual_labels'] = df_subset['womens_football']
df_subset['dict_labels'] = df_subset['womens_football.1']

In [25]:
df_subset.head()

,type,sectionName,webTitle,webUrl,headline,article_text,womens_football,word_count,author,char_count,...,article_text_split,womens_football.1,year,month,before_july_2022,month_year,live_blog,correct,manual_labels,dict_labels
0,article,Football,UK government's mixed messages casts shadow ov...,https://www.theguardian.com/football/2020/jun/...,UK government's mixed messages fail to diminis...,The last time English professional football go...,0,805,David Conn,4973,...,"['The', 'last', 'time', 'English', 'profession...",0,2020,6,1,2020-06-01,0,True,0,0
1,liveblog,Football,"Antony to be available for Manchester United, ...",https://www.theguardian.com/football/live/2023...,"Antony to be available for Manchester United, ...","Here’s Football Daily, to which you can sign u...",0,4625,John Brewin (later) & Yara El-Shaboury (earlier),25827,...,"['Here’s', 'Football', 'Daily,', 'to', 'which'...",1,2023,9,0,2023-09-01,1,False,0,1
2,article,Football,Tall tales: how my local and I became part of ...,https://www.theguardian.com/football/2023/jul/...,Tall tales: how my local and I became part of ...,"From time to time in this job, an offer of ext...",0,1000,Barry Glendenning,5892,...,"['From', 'time', 'to', 'time', 'in', 'this', '...",0,2023,7,0,2023-07-01,0,True,0,0
3,article,Football,"Football League: Gayle double sinks Stoke, Joz...",https://www.theguardian.com/football/2018/sep/...,"Football League: Gayle double sinks Stoke, Joz...",Championship Stoke’s troubled start to life in...,0,691,John Brewin,4182,...,"['Championship', 'Stoke’s', 'troubled', 'start...",0,2018,9,1,2018-09-01,0,True,0,0
4,article,Football,'Something needs to change': Willian the lates...,https://www.theguardian.com/football/2021/feb/...,'Something needs to change': Willian the lates...,Willian has revealed that he is the latest foo...,0,426,Nick Ames,2655,...,"['Willian', 'has', 'revealed', 'that', 'he', '...",0,2021,2,1,2021-02-01,0,True,0,0


In [26]:
# split subset into liveblog and non-liveblog
df_liveblog = df_subset[df_subset['live_blog'] == 1]

In [29]:
df_articles = df_subset[df_subset['live_blog'] == 0]

In [23]:
# Calculate Precision, Recall, and F1-Score
precision = precision_score(df_subset['manual_labels'], df_subset['dict_labels'])
recall = recall_score(df_subset['manual_labels'], df_subset['dict_labels'])
f1 = f1_score(df_subset['manual_labels'], df_subset['dict_labels'])

# Display the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

Precision: 0.7457627118644068
Recall: 0.6875
F1-Score: 0.7154471544715446


/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_s

In [28]:
# Calculate Precision, Recall, and F1-Score
precision = precision_score(df_liveblog['manual_labels'], df_liveblog['dict_labels'])
recall = recall_score(df_liveblog['manual_labels'], df_liveblog['dict_labels'])
f1 = f1_score(df_liveblog['manual_labels'], df_liveblog['dict_labels'])

# Display the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")


# there is a relatively high rate of false positives in the liveblog subset

Precision: 0.625
Recall: 0.9090909090909091
F1-Score: 0.7407407407407406


/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_s

In [30]:
# Calculate Precision, Recall, and F1-Score
precision = precision_score(df_articles['manual_labels'], df_articles['dict_labels'])
recall = recall_score(df_articles['manual_labels'], df_articles['dict_labels'])
f1 = f1_score(df_articles['manual_labels'], df_articles['dict_labels'])

# Display the results
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# for these articles, the dictionary is relatively good at avoiding false positives but misses many true positives resulting in a lot of false negatives

Precision: 0.7906976744186046
Recall: 0.6415094339622641
F1-Score: 0.7083333333333334


/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/emmastoklundlee/opt/anaconda3/envs/gensim_update/lib/python3.9/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_s

In [6]:
df.head()

,type,sectionName,webTitle,webUrl,headline,article_text,womens_football,word_count,author,char_count,date,article_text_split,womens_football.1,year,month,before_july_2022,month_year,live_blog
0,article,Football,UK government's mixed messages casts shadow ov...,https://www.theguardian.com/football/2020/jun/...,UK government's mixed messages fail to diminis...,The last time English professional football go...,0.0,805,David Conn,4973,2020-06-17,"['The', 'last', 'time', 'English', 'profession...",False,2020,6,1,2020-06-01,0
1,liveblog,Football,"Antony to be available for Manchester United, ...",https://www.theguardian.com/football/live/2023...,"Antony to be available for Manchester United, ...","Here’s Football Daily, to which you can sign u...",0.0,4625,John Brewin (later) & Yara El-Shaboury (earlier),25827,2023-09-29,"['Here’s', 'Football', 'Daily,', 'to', 'which'...",True,2023,9,0,2023-09-01,1
2,article,Football,Tall tales: how my local and I became part of ...,https://www.theguardian.com/football/2023/jul/...,Tall tales: how my local and I became part of ...,"From time to time in this job, an offer of ext...",0.0,1000,Barry Glendenning,5892,2023-07-01,"['From', 'time', 'to', 'time', 'in', 'this', '...",False,2023,7,0,2023-07-01,0
3,article,Football,"Football League: Gayle double sinks Stoke, Joz...",https://www.theguardian.com/football/2018/sep/...,"Football League: Gayle double sinks Stoke, Joz...",Championship Stoke’s troubled start to life in...,0.0,691,John Brewin,4182,2018-09-01,"['Championship', 'Stoke’s', 'troubled', 'start...",False,2018,9,1,2018-09-01,0
4,article,Football,'Something needs to change': Willian the lates...,https://www.theguardian.com/football/2021/feb/...,'Something needs to change': Willian the lates...,Willian has revealed that he is the latest foo...,0.0,426,Nick Ames,2655,2021-02-19,"['Willian', 'has', 'revealed', 'that', 'he', '...",False,2021,2,1,2021-02-01,0


### Seeing whether just keeping the womens articles that dont overlap with mens helps improve the scores

In [26]:
# make a new column for where womens_dictionary is true and mens_dictionary is true is set to 0, and only womens_dictionary is true is set to 1, and only mens_dictionary is true is set to 0, and where both are false is set to 0
guardian_annotated['women_only'] = guardian_annotated['womens_dictionary'] & ~guardian_annotated['mens_dictionary']

In [27]:
guardian_annotated['women_only'].value_counts()

women_only
0    463
1     36
Name: count, dtype: int64

In [28]:
# check number of true womens_dictionary are 1 in Annotation column
guardian_annotated[guardian_annotated['Annotation'] == '1']['women_only'].value_counts()

women_only
1    32
0    27
Name: count, dtype: int64